In [31]:
import sys
from pathlib import Path
from pprint import pprint

In [22]:
src_path = Path("../src").resolve()
print(src_path)
sys.path.append(str(src_path))

F:\OneDrive\github\analytics_api\src


In [42]:
from sqlmodel import Session, select
from api.db.session import engine
from api.events.models import EventModel
from timescaledb.hyperfunctions import time_bucket
from sqlalchemy import func
from datetime import datetime, timedelta, timezone 

In [ ]:
with Session(engine) as session:
    query = select(EventModel).order_by(EventModel.updated_at.asc()).limit(10)
    compiled_query = query.compile(compile_kwargs={"literal_binds": True})
    print(compiled_query) # to be run in PopSQL
    print("")
    print(str(query))
    # results = session.exec(query).fetchall()
    # pprint(results)

SELECT eventmodel.id, eventmodel.time, eventmodel.page, eventmodel.description, eventmodel.updated_at 
FROM eventmodel ORDER BY eventmodel.updated_at ASC
 LIMIT 10

SELECT eventmodel.id, eventmodel.time, eventmodel.page, eventmodel.description, eventmodel.updated_at 
FROM eventmodel ORDER BY eventmodel.updated_at ASC
 LIMIT :param_1


In [43]:

with Session(engine) as session:
    bucket = time_bucket("1 hour", EventModel.time)
    pages = ['/about', '/contact', '/pages', '/pricing']
    start = datetime.now(timezone.utc) - timedelta(hours=1)
    finish = datetime.now(timezone.utc) + timedelta(hours=1)
    query = (
        select(
            bucket,
            EventModel.page,
            func.count()    
        )
        .where(
            EventModel.time > start,
            EventModel.time <= finish,
            EventModel.page.in_(pages)
        )
        .group_by(
            bucket,
            EventModel.page
        )
        .order_by(
            bucket,
            EventModel.page
        )
    )
    compiled_query = query.compile(compile_kwargs={"literal_binds": True})
    print(compiled_query) # to be run in PopSQL
    print("")
    print(str(query))
    # results = session.exec(query).fetchall()
    # pprint(results)

SELECT time_bucket('1 hour'::interval, eventmodel.time) AS time_bucket_1, eventmodel.page, count(*) AS count_1 
FROM eventmodel 
WHERE eventmodel.time > '2025-10-28 22:26:55.045049+00:00' AND eventmodel.time <= '2025-10-29 00:26:55.045049+00:00' AND eventmodel.page IN ('/about', '/contact', '/pages', '/pricing') GROUP BY time_bucket('1 hour'::interval, eventmodel.time), eventmodel.page ORDER BY time_bucket('1 hour'::interval, eventmodel.time), eventmodel.page

SELECT time_bucket('1 hour'::interval, eventmodel.time) AS time_bucket_1, eventmodel.page, count(*) AS count_1 
FROM eventmodel 
WHERE eventmodel.time > :time_1 AND eventmodel.time <= :time_2 AND eventmodel.page IN (__[POSTCOMPILE_page_1]) GROUP BY time_bucket('1 hour'::interval, eventmodel.time), eventmodel.page ORDER BY time_bucket('1 hour'::interval, eventmodel.time), eventmodel.page
